In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import os

In [33]:
# Grab the directory of the current file
# Should be ./Project/src/Program
try:
    current_dir = Path(__file__).parent.absolute()
except NameError:
    # If using a Jupyter notebook
    try:
        current_dir = Path.cwd()
    except:
        # If continuing to fail set path manually
        current_dir = Path("D:/School/CS 434/Project/src/Program")

# Go to the Data directory 
# Should be ./Project/src/Data
data_dir = current_dir.parent / "Data"

# Get the Datasets path
dataset_path = data_dir / "Dataset.csv"

# Verify the file exists
if dataset_path.exists():
    print(f"Dataset found at: {dataset_path}")
    df = pd.read_csv(dataset_path)
else:
    print(f"Dataset not found at expected path: {dataset_path}")

Dataset found at: d:\School\CS 434\Project\src\Data\Dataset.csv


In [ ]:
# Check for any duplicate rows and remove them
df = df.drop_duplicates()

# Convert numeric comlumns to appropriate types
numeric_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Categorical columns
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']

In [ ]:
# Create first cleansed dataset - using median/mode imputation
